In [4]:
import shutil, subprocess, tempfile, sys
from pathlib import Path

# --- Locate src/data from a notebook in notebooks/ (siblings) ---
NB_DIR   = Path.cwd().resolve()
SRC_DIR  = NB_DIR.parent / "src"
DATA_DIR = SRC_DIR / "data"

print("Notebook CWD:", NB_DIR)
print("SRC_DIR     :", SRC_DIR)
print("DATA_DIR    :", DATA_DIR)

if not SRC_DIR.exists() or not DATA_DIR.exists():
    raise FileNotFoundError("Could not find sibling src/ and src/data/ next to notebooks/.")

# --- Files to test (must live in src/data) ---
mapping_files = [
    "bfo-mapping.ttl",
    "ies-mapping.ttl",
    "ccom-mapping.ttl",
    "qudt-mapping.ttl",
    "ccot-mapping.ttl",
    "time-mapping.ttl",  # keep this name if that's what your course uses
]

# --- Sanity: is ROBOT available? ---
# On Windows, try to find robot.bat or robot.exe
robot_cmd = shutil.which("robot")
if robot_cmd is None:
    # Try explicitly with .bat extension
    robot_cmd = shutil.which("robot.bat")
if robot_cmd is None:
    raise RuntimeError("ROBOT not found on PATH. Install ROBOT and make sure 'robot' or 'robot.bat' is runnable.")

print(f"Found ROBOT at: {robot_cmd}")

# Print robot version using shell=True on Windows
if sys.platform == "win32":
    subprocess.run(["robot", "--version"], check=False, shell=True)
else:
    subprocess.run([robot_cmd, "--version"], check=False)

reasoners = ["HermiT", "ELK"]

def run_robot_reason(input_file: Path, reasoner: str) -> bool:
    """Run robot reason with a temporary output; return True on success."""
    with tempfile.NamedTemporaryFile(suffix=".ttl", delete=True) as tmp:
        if sys.platform == "win32":
            # On Windows, use shell=True to handle .bat files
            res = subprocess.run(
                [
                    "robot", "reason",
                    "--reasoner", reasoner,
                    "--input", str(input_file),
                    "--output", tmp.name,
                ],
                capture_output=True,
                text=True,
                shell=True
            )
        else:
            res = subprocess.run(
                [
                    robot_cmd, "reason",
                    "--reasoner", reasoner,
                    "--input", str(input_file),
                    "--output", tmp.name,
                ],
                capture_output=True,
                text=True,
            )
    return res.returncode == 0

any_ran = False
for name in mapping_files:
    path = DATA_DIR / name
    if not path.exists():
        print(f"SKIP  (missing): {name}")
        continue
    any_ran = True
    for r in reasoners:
        ok = run_robot_reason(path, r)
        status = "SUCCESS" if ok else "FAILURE"
        print(f"{status}: {name} [{r}]")

if not any_ran:
    print("No mapping files found in src/data to test.")


Notebook CWD: C:\Users\crist\Documents\GitHub\Ontology-Tradecraft\Ontology-Tradecraft\projects\project-3\assignment\notebooks
SRC_DIR     : C:\Users\crist\Documents\GitHub\Ontology-Tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src
DATA_DIR    : C:\Users\crist\Documents\GitHub\Ontology-Tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src\data
Found ROBOT at: C:\WINDOWS\robot.BAT
FAILURE: bfo-mapping.ttl [HermiT]
FAILURE: bfo-mapping.ttl [ELK]
FAILURE: ies-mapping.ttl [HermiT]
FAILURE: ies-mapping.ttl [ELK]
FAILURE: ccom-mapping.ttl [HermiT]
FAILURE: ccom-mapping.ttl [ELK]
FAILURE: qudt-mapping.ttl [HermiT]
FAILURE: qudt-mapping.ttl [ELK]
SKIP  (missing): ccot-mapping.ttl
FAILURE: time-mapping.ttl [HermiT]
FAILURE: time-mapping.ttl [ELK]
